In [ ]:
import pandas as pd
import tqdm
from tqdm import tqdm

In [ ]:
type_list = ['INTP','ENTP','INTJ','ENTJ','ISTP','ESTP','ISTJ','ESTJ','INFP','ENFP','INFJ','ENFJ','ISFP','ESFP','ISFJ','ESFJ']
json_dir = 'drive/MyDrive/MBTI_DL_Data/RAW_JSON/'
csv_dir = 'drive/MyDrive/MBTI_DL_Data/Dataframes/'
img_dr = 'drive/MyDrive/MBTI_DL_Data/Images/'

In [ ]:
all_tweets_df = pd.DataFrame()
for p_type in tqdm(type_list):

  likes_df = pd.read_json(json_dir + p_type + '/' + p_type + '_likes_records.json').transpose()
  likes_df['liked_by'] = likes_df['liked_by'].apply(lambda x: x[0])

  likes_df.reset_index(drop=True, inplace=True)
  liked_by_set = likes_df.liked_by.unique()

  tweets_df = pd.read_json(json_dir + p_type + '/' + p_type + '_stats_records.json').transpose()
  tweets_df['creator_id'] = tweets_df['creator_id'].apply(lambda x: str(x))

  tweets_df.reset_index(drop=True, inplace=True)
  tweeted_by_set = tweets_df.creator_id.unique()

  follows_df = pd.read_json(json_dir + p_type + '/' + p_type + '_follows_records.json').transpose()
  base_info_df = pd.read_json(json_dir + 'mined_users.json').transpose()

  extravert = 1 if p_type[0] == 'E' else 0
  intuitive = 1 if p_type[1] == 'N' else 0
  thinking = 1 if p_type[2] == 'T' else 0
  judging = 1 if p_type[3] == 'J' else 0

  for i in liked_by_set:
      if (i not in tweeted_by_set) or (int(i) not in set(base_info_df.user_id)):
        continue
      temp_df = follows_df[follows_df['followed_by'].apply(lambda x: 1 if i in x else 0)==1]
      base_temp = base_info_df.loc[int(i)]
      all_tweets_df = all_tweets_df.append({'user_id' : i , 'follow_follows_counts': temp_df['follows_count'].values.tolist(),'own_bio': base_temp['bio'], 'profile_pic_url': base_temp['profile_pic_url'], 'profile_back_url': base_temp['profile_back_url'], 'follow_stats_counts': temp_df['statuses_count'].values.tolist(), 'follow_back_pic_urls': temp_df['profile_back_url'].values.tolist(), 'follow_profile_pic_urls': temp_df['profile_pic_url'].values.tolist(),  'follow_bios': temp_df['bio'].values.tolist(), 'liked_tweets': likes_df[likes_df['liked_by']==i]['text'].values.tolist(),'liked_media': likes_df[likes_df['liked_by']==i]['media'].values.sum(), 'type': p_type, 'extravert': extravert, 'intuitive': intuitive, 'thinking': thinking, 'judging': judging, 'statuses': tweets_df[tweets_df['creator_id']==i]['text'].values.tolist(),'own_media': tweets_df[tweets_df['creator_id']==i]['media'].values.sum(), 'lang': tweets_df[tweets_df['creator_id']==i]['lang'].values[0]}, ignore_index=True)

     # 'favorites_count': base_temp['favorites_count:'], 'follows_count': base_temp['follows_count'], 'stats_count': base_temp['statuses_count'], 




100%|██████████| 16/16 [11:10<00:00, 41.90s/it]


In [ ]:

#save1 = all_tweets_df.copy()

In [100]:
df = pd.read_csv(csv_dir + 'master_v1.0.csv', index_col=0)
base_info_df = pd.read_json(json_dir + 'mined_users.json').transpose()

In [102]:
base_info_df

,screen_name,name,bio,user_id,favorites_count:,statuses_count,follows_count,profile_back_url,profile_pic_url,queried_at
104066691,M40A3Predator,Logan Keesling,College student. INTP.,104066691,1650,6669,400,http://abs.twimg.com/images/themes/theme9/bg.gif,http://pbs.twimg.com/profile_images/1031267728...,2021-05-08 00:08:50.658936
728747154,LukeDashjr,Luke Dashjr,"Roman #Catholic*, husband, father of 7 childre...",728747154,32648,58883,139,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1299396704...,2021-05-08 00:08:50.671926
321645158,Sacheverell,Sacheverell,Trust & Safety Lead @vrchat. | Formerly T&S @w...,321645158,32461,46164,1274,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9959137055...,2021-05-08 00:08:50.678938
346419306,Blondiettv,Shan,tired. | @Twitch Partner |,346419306,24435,24809,1092,http://abs.twimg.com/images/themes/theme19/bg.gif,http://pbs.twimg.com/profile_images/1377152197...,2021-05-08 00:08:50.688448
709345926,AlexisWolfy,AlexWolfy,24 | artist | sonic and mario are pretty cool ...,709345926,37507,6800,435,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1382724888...,2021-05-08 00:08:50.696464
...,...,...,...,...,...,...,...,...,...,...
590000506,cherilnaey,CN너이 | ꪔ̤̮,ESFJ | เหนยน่ะ : my euphoria jungkook❥ EN/TH O...,590000506,30834,40699,303,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/1302761904...,2021-05-08 00:16:12.807363
884119350445305856,moh_q6,‏كارلــــوس ❣︎ 😌,"Cute person, gamer , I am 16 years old .| I lo...",884119350445305856,14631,18694,54,None,http://pbs.twimg.com/profile_images/1380068998...,2021-05-08 00:16:13.004061
15939890,chubacca,Alyson Chu,"ESFJ, books, food, feminism, and YouTube. Reci...",15939890,188,1985,228,http://abs.twimg.com/images/themes/theme5/bg.gif,http://pbs.twimg.com/profile_images/7166585412...,2021-05-08 00:16:13.203962
757759104,SunnyDeKnight,Sunny ~ ☼,"[Esp/Eng] 26. ♀. In love with Anime/Manga, vid...",757759104,6384,100502,361,http://abs.twimg.com/images/themes/theme4/bg.gif,http://pbs.twimg.com/profile_images/1290606759...,2021-05-08 00:16:13.397851


In [106]:
base_info_df['user_id'] = base_info_df['user_id'].apply(lambda x: int(x))

In [107]:
df = df.merge(base_info_df, how= 'inner', left_on='user_id', right_on='user_id')

In [ ]:
df.loc[4,'profile_pic_url']

'http://pbs.twimg.com/profile_images/1382724888952242178/gPDiG88o_normal.jpg'

In [110]:
df.head()

,extravert,follow_back_pic_urls,follow_bios,follow_follows_counts,follow_profile_pic_urls,follow_stats_counts,intuitive,judging,lang,liked_media,liked_tweets,own_media,statuses,thinking,type,user_id,screen_name,name,bio,favorites_count:,statuses_count,follows_count,profile_back_url,profile_pic_url,queried_at
0,0.0,['http://abs.twimg.com/images/themes/theme1/bg...,['Leader; follower. Reader; writer. Speaker; l...,"[118, 1055, 3, 1989, 69, 335, 654, 980, 98, 39...",['http://pbs.twimg.com/profile_images/64760639...,"[87848, 3561, 1468, 54864, 276, 5255, 116451, ...",1.0,0.0,und,['https://pbs.twimg.com/media/Em-45V2VkAAUJqY....,['@M40A3Predator hope you been well. Miss you ...,['https://pbs.twimg.com/media/D4dqPJLVUAAo3Fk....,"['https://t.co/lkHBulmhvc', 'We evolved the ap...",1.0,INTP,104066691,M40A3Predator,Logan Keesling,College student. INTP.,1650,6669,400,http://abs.twimg.com/images/themes/theme9/bg.gif,http://pbs.twimg.com/profile_images/1031267728...,2021-05-08 00:08:50.658936
1,0.0,['http://abs.twimg.com/images/themes/theme1/bg...,['Making people proud to care about gun rights...,"[67, 414, 1024, 62, 1, 68, 1703, 681, 85, 4030...",['http://pbs.twimg.com/profile_images/10861406...,"[2670, 3487, 30848, 5435, 1229, 10775, 3189, 2...",1.0,0.0,en,['https://pbs.twimg.com/media/E0Aq-wPXIAYhM28....,"[""@hackaday It was 1989. Someone changed the r...",[],"[""So USA has too many #COVID19 vaccines, while...",1.0,INTP,728747154,LukeDashjr,Luke Dashjr,"Roman #Catholic*, husband, father of 7 childre...",32648,58883,139,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1299396704...,2021-05-08 00:08:50.671926
2,0.0,['http://abs.twimg.com/images/themes/theme9/bg...,['#FreeMarket think tank advancing real soluti...,"[14892, 31, 1456, 2244, 874, 11342, 11, 71, 14...",['http://pbs.twimg.com/profile_images/13777942...,"[67741, 858, 59783, 89244, 9986, 10429, 136, 2...",1.0,0.0,en,['https://pbs.twimg.com/media/E0DO_ihWEAANspm....,"[""Patch V1.3.0\n\nWe've been listening, and we...",['https://pbs.twimg.com/media/EzTmWO8UUAE-cK-....,"['Life isn\'t fair.\n\nThat\'s not a lament, t...",1.0,INTP,321645158,Sacheverell,Sacheverell,Trust & Safety Lead @vrchat. | Formerly T&S @w...,32461,46164,1274,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9959137055...,2021-05-08 00:08:50.678938
3,0.0,"[None, None, 'http://abs.twimg.com/images/them...",['NAME will be announced closer to launch - HE...,"[1, 1376, 6122, 601, 1299, 690, 785, 340, 1079...",['http://pbs.twimg.com/profile_images/13842164...,"[6, 8753, 9855, 13930, 36826, 131, 2888, 9751,...",1.0,0.0,en,['https://pbs.twimg.com/amplify_video_thumb/12...,['It’s The Oscars tonight! I wasn’t invited. W...,['https://pbs.twimg.com/ext_tw_video_thumb/138...,"['Happy puppies 🥺🥰 https://t.co/O6FOXQQFtF', '...",1.0,INTP,346419306,Blondiettv,Shan,tired. | @Twitch Partner |,24435,24809,1092,http://abs.twimg.com/images/themes/theme19/bg.gif,http://pbs.twimg.com/profile_images/1377152197...,2021-05-08 00:08:50.688448
4,0.0,"[None, 'http://abs.twimg.com/images/themes/the...",['Occasional funny clips from games I recorded...,"[2, 687, 37, 408, 360, 765, 975, 239, 360, 492...",['http://pbs.twimg.com/profile_images/13360189...,"[2, 0, 6557, 1348, 2898, 6999, 37687, 1219, 89...",1.0,0.0,en,['https://pbs.twimg.com/media/EwkPYfNVIAQU7Vs....,"['""Oh I\'m not very good at that game, but sur...",['https://pbs.twimg.com/media/Ew4wixCW8AEBwCn....,['long time no see\nI started a new job last w...,1.0,INTP,709345926,AlexisWolfy,AlexWolfy,24 | artist | sonic and mario are pretty cool ...,37507,6800,435,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1382724888...,2021-05-08 00:08:50.696464


In [111]:
df.to_csv(csv_dir + 'master_v2.0.csv')

**Images**

In [ ]:
import requests

In [ ]:
for user_id in tqdm(df.index.values.tolist()[:5]):
   
   id, profile_access_url = df.loc[user_id, ['user_id', 'profile_pic_url']]
   print(profile_access_url)
   img_type = '.' + profile_access_url.split('.')[-1]

   profile_access_url = profile_access_url.split("_normal")[0] + img_type
   print(profile_access_url)
   print(img_type)

100%|██████████| 5/5 [00:00<00:00, 545.59it/s]

http://pbs.twimg.com/profile_images/1031267728349310976/As9042ef_normal.jpg
http://pbs.twimg.com/profile_images/1031267728349310976/As9042ef.jpg
.jpg
http://pbs.twimg.com/profile_images/1299396704672739328/S9T_Gzd6_normal.jpg
http://pbs.twimg.com/profile_images/1299396704672739328/S9T_Gzd6.jpg
.jpg
http://pbs.twimg.com/profile_images/995913705589112832/oKqcUkaG_normal.jpg
http://pbs.twimg.com/profile_images/995913705589112832/oKqcUkaG.jpg
.jpg
http://pbs.twimg.com/profile_images/1377152197381656584/2nl6JzLk_normal.jpg
http://pbs.twimg.com/profile_images/1377152197381656584/2nl6JzLk.jpg
.jpg
http://pbs.twimg.com/profile_images/1382724888952242178/gPDiG88o_normal.jpg
http://pbs.twimg.com/profile_images/1382724888952242178/gPDiG88o.jpg
.jpg


In [ ]:
for user_id in tqdm(df.index.values.tolist()):
   
   id, profile_access_url = df.loc[user_id, ['user_id', 'profile_pic_url']]
   img_type = '.' + profile_access_url.split('.')[-1]

   profile_access_url = profile_access_url.split("_normal")[0] + img_type
  
   profile_save_url = img_dr + 'profile_pics/' + str(id) + img_type

   profile_img_data = requests.get(profile_access_url).content
   if not (profile_img_data == None):

      with open(profile_save_url, 'wb') as handler:
         handler.write(profile_img_data)

100%|██████████| 3857/3857 [05:03<00:00, 12.69it/s]


In [ ]:
for user_id in tqdm(df.index.values.tolist()):
   
   id, profile_access_url = df.loc[user_id, ['user_id', 'profile_back_pic_url']]
   img_type = '.' + profile_access_url.split('.')[-1]

   if (img_type == '.gif'):
     continue

   profile_access_url = profile_access_url.split("_normal")[0] + img_type
  
   profile_save_url = img_dr + 'profile_back_pics/' + str(id) + img_type

   profile_img_data = requests.get(profile_access_url).content
   if not (profile_img_data == None):

      with open(profile_save_url, 'wb') as handler:
         handler.write(profile_img_data)

  0%|          | 0/3857 [00:00<?, ?it/s]


FileNotFoundError: ignored

In [ ]:
https://twitter.com/naval/photo

In [ ]:
all_tweets_df

n1_df = all_tweets_df.copy()


In [ ]:
new_stats_df = pd.DataFrame()
for p_type in tqdm(type_list):

  print(p_type)
  status_df = pd.read_json(json_dir + p_type + '/' + p_type + '_stats_records.json').transpose()
  status_df['creator_id'] = status_df['creator_id'].apply(lambda x: str(x))

  status_df.reset_index(drop=True, inplace=True)
  written_by_set = status_df.creator_id.unique()


#  extravert = 1 if p_type[0] == 'E' else 0
#  intuitive = 1 if p_type[1] == 'N' else 0
#  thinking = 1 if p_type[2] == 'T' else 0
#  judging = 1 if p_type[3] == 'J' else 0

  for i in written_by_set:
      new_stats_df = new_stats_df.append({'liked_by' : i , 'statuses': str(status_df[status_df['creator_id']==i]['text'].sum()),'own_media': status_df[status_df['creator_id']==i]['media'].sum(), 'type': p_type, 'lang': status_df[status_df['creator_id']==i]['lang'].sum() }, ignore_index=True)






  0%|          | 0/16 [00:00<?, ?it/s]

INTP







  6%|▋         | 1/16 [00:06<01:30,  6.04s/it]

ENTP







 12%|█▎        | 2/16 [00:12<01:26,  6.16s/it]

...






 88%|████████▊ | 14/16 [01:10<00:09,  4.66s/it]

ISFJ







 94%|█████████▍| 15/16 [01:16<00:05,  5.04s/it]

ESFJ







100%|██████████| 16/16 [01:20<00:00,  5.00s/it]


In [ ]:
new_stats_df.columns = ['lang', 'own_media', 'statuses', 'type', 'liked_by']

In [ ]:
syn1_df = pd.merge(new_df, new_stats_df, on = ['liked_by'] )

In [ ]:
syn1_df.head()

,extravert,intuitive,judging,liked_by,media,text,thinking,type_x,lang,own_media,statuses,type_y
0,0.0,1.0,0.0,104066691,@M40A3Predator hope you been well. Miss you my broKaai 08 Reacts to CoD4 Dualtage Montage- M40A3...,"['https://pbs.twimg.com/media/Em-45V2VkAAUJqY.jpg', 'https://pbs.twimg.com/amplify_video_thumb/1...",1.0,INTP,undenundenen,"[https://pbs.twimg.com/media/D4dqPJLVUAAo3Fk.jpg, https://pbs.twimg.com/media/Dx4DkD7U0AAQxYd.jpg]",https://t.co/lkHBulmhvcWe evolved the apoe3 genotype to better deal with metabolizing saturated ...,INTP
1,0.0,1.0,0.0,728747154,@hackaday It was 1989. Someone changed the root password on a shared SunOS SPARCstation over th...,"['https://pbs.twimg.com/media/E0Aq-wPXIAYhM28.jpg', 'https://pbs.twimg.com/media/E0AHnYuWUAMxy33...",1.0,INTP,enenenen,[],"So USA has too many #COVID19 vaccines, while other countries are begging for them.\n\nThis isn't...",INTP
2,0.0,1.0,0.0,321645158,"Patch V1.3.0\n\nWe've been listening, and we've been busy. With this update, we have addressed c...","['https://pbs.twimg.com/media/E0DO_ihWEAANspm.jpg', 'https://pbs.twimg.com/media/E0C9IrRUYAIUt7V...",1.0,INTP,enenenenenenenenenenenenenenenenenenenenenenenenenenenenenenundenenen,"[https://pbs.twimg.com/media/EzTmWO8UUAE-cK-.png, https://pbs.twimg.com/media/EyqVbdYXIAAaf2K.pn...","Life isn't fair.\n\nThat's not a lament, that's a fact. ""Fair"" is a nonexistent concept, like ""n...",INTP
3,0.0,1.0,0.0,346419306,It’s The Oscars tonight! I wasn’t invited. Was it something I said? 😂 https://t.co/P87Uau4D9uDea...,"['https://pbs.twimg.com/amplify_video_thumb/1214004378790531073/img/KNZKuxf5vWkjL9Yr.jpg', 'http...",1.0,INTP,enenenenenenenenenenenenenenenenenenenenundenenhtenenenenentrundenenenenenenenenenenenenenetenen,"[https://pbs.twimg.com/ext_tw_video_thumb/1384623936667852801/pu/img/WnRz3SVgoqqiQWxs.jpg, https...",Happy puppies 🥺🥰 https://t.co/O6FOXQQFtFI’m actually streaming tonight. 9pm ESTI can’t believe t...,INTP
4,0.0,1.0,0.0,709345926,"""Oh I'm not very good at that game, but sure, I'll play with you"" https://t.co/u2jFwYKUFzThe onl...","['https://pbs.twimg.com/media/EwkPYfNVIAQU7Vs.jpg', 'https://pbs.twimg.com/media/Ez2GbtaWQAE9qMx...",1.0,INTP,enenenenenenenen,"[https://pbs.twimg.com/media/Ew4wixCW8AEBwCn.jpg, https://pbs.twimg.com/media/EvvP30qXEAENmNs.jp...",long time no see\nI started a new job last week and I’ve had to hella adjust my schedule. it’ll ...,INTP


In [ ]:
new_follows_df = pd.DataFrame()
for p_type in tqdm(type_list[4:5]):

  print(p_type)
  status_df = pd.read_json(json_dir + p_type + '/' + p_type + '_follows_records.json').transpose()
  new_follows_df = new_follows_df.append(status_df)


print(new_follows_df.columns)
n1_df[['following', 'follow_bios', 'statuses_count', 'follows_count', 'profile_back_urls', 'profile_pic_urls']] = ''
new_follows_df.head()

#  status_df.reset_index(drop=True, inplace=True)
#  written_by_set = status_df.user_id.unique()


#  extravert = 1 if p_type[0] == 'E' else 0
#  intuitive = 1 if p_type[1] == 'N' else 0
#  thinking = 1 if p_type[2] == 'T' else 0
#  judging = 1 if p_type[3] == 'J' else 0

 # for i in liked_by_set:
 #     new_follows_df = new_follows_df.append({'written_by' : i , 'statuses': str(likes_df[likes_df['written_by']==i]['text'].sum()),'own_media': likes_df[likes_df['written_by']==i]['media'].sum(), 'type': p_type, 'lang': likes_df[likes_df['written_by']==i]['lang'].sum() }, ignore_index=True)




  0%|          | 0/1 [00:00<?, ?it/s]

ISTP





100%|██████████| 1/1 [00:00<00:00,  1.79it/s]

Index(['followed_by', 'screen_name', 'name', 'bio', 'favorites_count:',
       'statuses_count', 'follows_count', 'profile_back_url',
       'profile_pic_url', 'language', 'queried_at'],
      dtype='object')


,followed_by,screen_name,name,bio,favorites_count:,statuses_count,follows_count,profile_back_url,profile_pic_url,language,queried_at
155688134,[56973040],jwaaaap,Jan Willem Nijman,"Game designer: Disc Room, Minit Franchise, Nuc...",75512,41185,5075,http://abs.twimg.com/images/themes/theme11/bg.gif,http://pbs.twimg.com/profile_images/1301196144...,None,2021-04-24 16:17:33.085657
223056491,[56973040],bob_groothuis,Bob Groothuis,"HDR panorama photography for Movies, VFX, 3D, ...",37,284,254,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1358403938...,None,2021-04-24 16:17:33.085657
822180716175917056,[56973040],BioGridGame,BioGrid,There aren't enough ecology sims out there. I ...,932,389,309,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8221886095...,None,2021-04-24 16:17:33.085657
64861050,[56973040],zellah,Daniel Zeller,Fur tool: https://t.co/sOBq84nB0x\nAndroid & U...,2757,822,711,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/1026078358...,None,2021-04-24 16:17:33.085657
1372757878017196032,[56973040],ArcticAwakening,Arctic Awakening,An episodic narrative adventure set after a pl...,83,16,6,None,http://pbs.twimg.com/profile_images/1372961543...,None,2021-04-24 16:17:33.085657


In [ ]:
n1_df

,extravert,intuitive,judging,lang,liked_media,liked_tweets,own_media,statuses,thinking,type,user_id,following,follow_bios,statuses_count,follows_count,profile_back_urls,profile_pic_urls
0,0.0,0.0,0.0,en,[https://pbs.twimg.com/tweet_video_thumb/Ezjpk...,"[In Unity, on a multi-material mesh, have you ...",[https://pbs.twimg.com/ext_tw_video_thumb/1382...,[Practically all my assets are currently 50% o...,1.0,ISTP,56973040,jwaaaapbob_groothuisBioGridGamezellahArcticAwa...,"Game designer: Disc Room, Minit Franchise, Nuc...",,,,http://pbs.twimg.com/profile_images/1301196144...
1,0.0,0.0,0.0,en,[https://pbs.twimg.com/media/EjvxdiwUYAAzWor.j...,[Just staying Covid-safe wearing by the flayed...,[https://pbs.twimg.com/tweet_video_thumb/EsTga...,[#science Twitter! I am looking for recommenda...,1.0,ISTP,342774244,MakeProjects_MarkRoberRubeGoldbergDiscoveryEdA...,"We are a community of over 25,000 Problem Solv...",,,,http://pbs.twimg.com/profile_images/1252618733...
2,0.0,0.0,0.0,en,[https://pbs.twimg.com/media/Ezvh7bVUUAAEPWV.j...,[A big thank you to everyone who came out yest...,[https://pbs.twimg.com/media/Ezgmp1SXoAMj29x.j...,[A few years ago I bought this art print and j...,1.0,ISTP,20994967,C9ViennayenkoesUsagiKira_Dahlia_PennaAndyLuniq...,💫Cosmic Dragon - May 2021💫 Vtuber for @Cloud9 ...,,,,http://pbs.twimg.com/profile_images/1368475955...
3,0.0,0.0,0.0,en,[https://pbs.twimg.com/ext_tw_video_thumb/1347...,[SeaShantyTok keeps getting better https://t.c...,[],"[Y'all, I am so happy that you share your vacc...",1.0,ISTP,42466620,POTUSZachSmadutnwhiskeywomanOnThisDayShethelos...,"46th President of the United States, husband t...",,,,http://pbs.twimg.com/profile_images/1380530524...
4,0.0,0.0,0.0,en,[https://pbs.twimg.com/media/Ezr8X5-X0AogpkM.j...,[EXO is the main character in kpop stan twt. h...,[https://pbs.twimg.com/media/Ezte4weUYAYWs6s.jpg],"[My wifi is not connecting today wtf 🙃, No bas...",1.0,ISTP,287196332,zeffirelliiihourlyleejuyeonescobarfedmylifesme...,"later! “I dont hate you, I’m just not necessar...",,,,http://pbs.twimg.com/profile_images/1384598993...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.0,0.0,0.0,en,[https://pbs.twimg.com/media/EzxoXT6XoAMVZ-G.j...,[“It hurts my feelings every time you bring up...,[],[This isn't even Williamsburg but imagine the ...,1.0,ISTP,159245855,sadgalsunnymamayambilidennisofthesandOfficialK...,ﾟ+..｡*ﾟ+ ｡⋆ʚ♡⃛ɞ i make music @sad_x_china ♪ 🦋...,,,,http://pbs.twimg.com/profile_images/1385915963...
100,0.0,0.0,0.0,es,[https://pbs.twimg.com/ext_tw_video_thumb/1386...,[→ sᴏᴍᴇ sᴛᴜғғ\n ↳ https://t.co/5p5fIF7MRF h...,[],[Lo maravilloso que es este vídeo??? Adoro htt...,1.0,ISTP,1366377464,rhodeslibbysluchii_liciousnotcuteaf_luviegguks...,ʙᴛꜱ ♡ ᴀɴɪᴍᴇツ「I know you're not in love like y...,,,,http://pbs.twimg.com/profile_images/1384783605...
101,0.0,0.0,0.0,en,[https://pbs.twimg.com/media/EzrhIPjX0AQLe_0.j...,[“Izuku pls pose for the picture”\n\n*internal...,[https://pbs.twimg.com/ext_tw_video_thumb/1379...,[Not me going to pick up the things in our PO ...,1.0,ISTP,155935581,chaookamijoel_harlowlaovaansorensodiumCosplays...,I cosplay and study biology. You can find my c...,,,,http://pbs.twimg.com/profile_images/1385396860...
102,0.0,0.0,0.0,en,[https://pbs.twimg.com/media/ExWyYV0WUAcXeS1.p...,[#WritingCommunity you know how I always say “...,[https://pbs.twimg.com/media/EwOS1S1UcAEHD9r.p...,[Top Ten Tuesday: Colorful Book Covers https:/...,1.0,ISTP,4745024906,JonesLakaiikathryninman9W_Angels_Wingsleahjord...,"Follower of Jesus, School Psychologist, Writer...",,,,http://pbs.twimg.com/profile_images/1264014764...


In [ ]:
save_master_file = open(csv_dir  + 'MasterFrame.csv')
syn1_df.to_csv(save_master_file)

In [ ]:
pd.set_option("display.max_rows", None)
status_df[status_df['followed_by'].apply(lambda x: 1 if len(x)>3 else 0)==1]


,followed_by,screen_name,name,bio,favorites_count:,statuses_count,follows_count,profile_back_url,profile_pic_url,language,queried_at
1323730225067339776,"[784984, 33635755, 29941547, 728747154, 14339685]",WhiteHouse,The White House,Welcome to the Biden-Harris White House! Tweets may be archived: https://t.co/IURuMIrzxb,0,891,5,None,http://pbs.twimg.com/profile_images/1351938473142448133/JQT93Cjo_normal.jpg,None,2021-04-28 09:00:16.480643
754006735468261376,"[1391299076, 1074006956128591873, 148222883, 1191084882002210816, 17915737, 186899369]",LilNasX,nope 🏹,“MONTERO (CALL ME BY YOUR NAME)” OUT NOW! ☎️❣️ https://t.co/QK0ddAoeGt,41704,9147,767,None,http://pbs.twimg.com/profile_images/1373007512522297345/aL4Ry-kc_normal.jpg,None,2021-04-28 09:16:52.095665
1356958547603513344,"[18059134, 8088362, 14353681, 15231550]",SecretaryPete,Secretary Pete Buttigieg,19th Secretary of Transportation. Working to ensure safety and make transportation more equitabl...,100,365,123,None,http://pbs.twimg.com/profile_images/1356958934532304903/muuwKYND_normal.jpg,None,2021-04-28 09:17:13.120846
1349154719386775552,"[976948512503029760, 29941547, 12443022, 18059134, 1870500607, 175419846, 17552125]",FLOTUS,Jill Biden,First Lady of the United States Jill Biden. Community college educator. Military mother. Grandmo...,1,92,4,None,http://pbs.twimg.com/profile_images/1349868692411211777/fE-VHwMl_normal.jpg,None,2021-04-28 09:17:13.120846
1349175483989188608,"[18569745, 18059134, 22278898, 1870500607, 17552125]",SecondGentleman,Douglas Emhoff,Second Gentleman of the United States. Devoted dad. Proud husband to @VP Kamala Harris.,0,95,6,None,http://pbs.twimg.com/profile_images/1349826378938675202/0DX4iKqh_normal.jpg,None,2021-04-28 09:17:13.120846
2178012643,"[1461354240, 46020493, 925361, 18968910]",balajis,balajis.com,"Immutable money, infinite frontier, eternal life. #Bitcoin",109913,16735,3829,http://abs.twimg.com/images/themes/theme9/bg.gif,http://pbs.twimg.com/profile_images/1363379554002165765/q6OIJhI6_normal.jpg,None,2021-04-28 09:17:38.348372
1349149096909668352,"[262364734, 7305042, 784984, 14613642, 976948512503029760, 29941547, 1870500607, 33391247, 18569...",POTUS,President Biden,"46th President of the United States, husband to @FLOTUS, proud dad & pop. Tweets may be archived...",0,598,12,None,http://pbs.twimg.com/profile_images/1380530524779859970/TfwVAbyX_normal.jpg,None,2021-04-28 09:50:29.988648
1349170292564905984,"[254274052, 22528795, 105838459, 33391247, 12443022, 17552125]",PressSec,Jen Psaki,WH Press Sec. Working mom of 2 small humans. Spends most days communicating about @POTUS' agenda...,55,141,232,None,http://pbs.twimg.com/profile_images/1349733243956785154/Hie5CWNw_normal.jpg,None,2021-04-28 09:50:29.988648
803694179079458816,"[7305042, 33635755, 33391247, 18569745, 12443022, 9589342, 1870500607, 14613642, 17552125]",VP,Vice President Kamala Harris,Vice President of the United States. Wife to the first @SecondGentleman. Momala. Auntie. Fightin...,1085,6047,6,None,http://pbs.twimg.com/profile_images/1380657780865044480/BBxmnji2_normal.jpg,None,2021-04-28 09:50:29.988648
2569743630,"[15604422, 33635755, 33391247, 19786800, 22278898]",TheAmandaGorman,Amanda Gorman,Poet,4656,3492,626,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1251186852805005312/Nn9KM9Fy_normal.jpg,None,2021-04-28 10:08:51.785966


In [ ]:
status_df.iloc[520445177]

IndexError: ignored

In [ ]:
sum = 0

for p_type in tqdm(type_list):

  print(p_type)
  likes_df = pd.read_json(json_dir + p_type + '/' + p_type + '_likes_records.json').transpose()
  likes_df['liked_by'] = likes_df['liked_by'].apply(lambda x: int(x[0]))

  likes_df.reset_index(drop=True, inplace=True)
  liked_by_set = set(likes_df['liked_by'].unique())
  sum += len(liked_by_set)

sum

  0%|          | 0/16 [00:00<?, ?it/s]

INTP


  6%|▋         | 1/16 [00:32<08:09, 32.63s/it]

ENTP


 12%|█▎        | 2/16 [01:05<07:37, 32.69s/it]

...

ISFJ


 94%|█████████▍| 15/16 [07:00<00:28, 28.23s/it]

ESFJ


100%|██████████| 16/16 [07:23<00:00, 27.72s/it]


3920

In [ ]:
len(set(likes_df['liked_by']))

209

In [ ]:
new_df

,extravert,intuitive,judging,liked_by,media,text,thinking,type
0,0.0,1.0,0.0,9.745463e+17,a normal person explains what’s happening on the stock market: https://t.co/zKKvULCirXhttps://t....,"['https://pbs.twimg.com/ext_tw_video_thumb/1354496485878972416/pu/img/Q36NubJuAfxbIWzn.jpg', 'ht...",1.0,INTP
1,0.0,1.0,0.0,9.200336e+17,IT’S THE SHADOW OF THE HELICOPTER FLYING ON ANOTHER WORLD https://t.co/JUUVId9OLrno bc how the f...,"['https://pbs.twimg.com/media/EzVNwekVgAIju8e.jpg', 'https://pbs.twimg.com/media/EzmbJ0jXMAwVYoN...",1.0,INTP
2,0.0,1.0,0.0,1.191085e+18,@HoodedMask @BabyBamBURR @Six_6ix_ @lillianscafe17 @Drebae_ Or you’re just dumb because thats th...,"['https://pbs.twimg.com/media/EzdYlsrXMAIVlm4.jpg', 'https://pbs.twimg.com/tweet_video_thumb/Ezc...",1.0,INTP
3,0.0,1.0,0.0,9.133011e+17,Just as India sent assistance to the United States as our hospitals were strained early in the p...,"['https://pbs.twimg.com/media/Ezw-vwTWQAsKq_D.jpg', 'https://pbs.twimg.com/media/EzbyLEhX0AQBM1_...",1.0,INTP
4,0.0,1.0,0.0,2.542741e+08,Today is The Day my brother died. This year more than half a million other families will mark Th...,"['https://pbs.twimg.com/media/E0EAIMOVcAY_LIl.jpg', 'https://pbs.twimg.com/media/E0D2plAX0AIsv9X...",1.0,INTP
...,...,...,...,...,...,...,...,...
3915,1.0,0.0,1.0,1.686988e+07,adults are just kids that moved to an apartmentLet's normalize not needing everything to be norm...,"['https://pbs.twimg.com/media/Evfw3tkXMAUCGEf.jpg', 'https://pbs.twimg.com/amplify_video_thumb/1...",0.0,ESFJ
3916,1.0,0.0,1.0,1.608350e+09,@Marathonswift A movie I actually watched before you! Disposable and unoriginal but entertaining...,"['https://pbs.twimg.com/media/Ez_0yLWXsAYnvSO.jpg', 'https://pbs.twimg.com/media/Ez5-CWVX0AAbRgV...",0.0,ESFJ
3917,1.0,0.0,1.0,5.250847e+08,رايتشل تفاجئ معجبين المسلسل ب Central perk ♥️😂 https://t.co/34Et6D9w4iأول كوب بعد الفطور:\nقهْوَ...,"['https://pbs.twimg.com/ext_tw_video_thumb/1387081594658242567/pu/img/yPHpv5DRAJPRGn8t.jpg', 'ht...",0.0,ESFJ
3918,1.0,0.0,1.0,1.326505e+08,"@nathanalbert Yes, and I think they bonded over their shared insight into being divine.Who will ...","['https://pbs.twimg.com/media/EztHl5YX0AEc3bp.jpg', 'https://pbs.twimg.com/media/EznaGpMWQAQ_MRX...",0.0,ESFJ


3903
